# 0518 Balance data

In [1]:
import pandas as pd
import numpy as np
import random
import time
time_start = time.time()

from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

PATH= "C:/Users/user/Ming/Final/"

In [2]:
y_train= pd.read_pickle(PATH+ "y_train.pkl")
X_train= pd.read_pickle(PATH+ "X_train.pkl")

In [3]:
group_loc= []
for i in range(1, 41) :
    loc= []
    
    for j in range(len(y_train)):
        if (y_train[j] == i) :
            loc.append(j)
            
    print(i, " Len: ", len(loc))
    group_loc.append(loc)

1  Len:  99928
2  Len:  88854
3  Len:  64269
4  Len:  58830
5  Len:  53435
6  Len:  39380
7  Len:  40830
8  Len:  30052
9  Len:  29669
10  Len:  25622
11  Len:  22693
12  Len:  22700
13  Len:  20563
14  Len:  14118
15  Len:  16391
16  Len:  10761
17  Len:  10528
18  Len:  10371
19  Len:  9168
20  Len:  8691
21  Len:  8627
22  Len:  7507
23  Len:  7211
24  Len:  5694
25  Len:  5352
26  Len:  4513
27  Len:  3920
28  Len:  3823
29  Len:  3560
30  Len:  3461
31  Len:  2323
32  Len:  2281
33  Len:  2015
34  Len:  1530
35  Len:  1447
36  Len:  1431
37  Len:  1182
38  Len:  1238
39  Len:  1092
40  Len:  901


# Sampling

In [4]:
sample= []
n= 1500

for i in range(40):
    if (len(group_loc[i]) > n) :
        sample.extend(random.sample(group_loc[i], n))
    else :
        sample.extend(group_loc[i])       

print(len(sample))

58291


In [5]:
X_train_sam= X_train[sample]
y_train_sam= np.array(y_train)[sample]

In [8]:
X_train_sam.dump(PATH+ "X_train_sam.pkl")
y_train_sam.dump(PATH+ "y_train_sam.pkl")

# Words frequence

In [75]:
words= []
for i in data.index :
    words.extend(data["Sentence"][i])

words = pd.DataFrame(words)
words = pd.DataFrame(words[0].value_counts())
words.columns= [filename]
words.insert(0, 'Word', words.index)
words.to_pickle(PATH+ filename+ "words.pkl")
words

,Word,Sam6_
是,是,3657
好,好,3413
一,一,3036
加油,加油,2935
真的,真的,2483
...,...,...
學生證,學生證,1
做人,做人,1
綿密,綿密,1
原生,原生,1


# One hot encode

In [76]:
words= list(words['Word'])
oh_encode_df= pd.get_dummies(list(set(words)))

del words

loc= []
for i in oh_encode_df.columns :
    for j in oh_encode_df.index :
        if (oh_encode_df[i][j] == 1) :
            loc.append(j)
            break

oh_encode_df= oh_encode_df.loc[loc]
oh_encode_df= oh_encode_df.reset_index(drop= True)

del loc

In [77]:
oh= []

for i in data.index :
    oh.append(oh_encode_df[data["Sentence"][i]].T.sum().clip(0,1).values)
    #if (i % 10000 == 0) :
        #time_end = time.time() 
        #timespend = (time_end- time_start)/60
        #print("Done ", i, "時間: ", round(timespend, 2), " 分鐘")
    
oh= pd.DataFrame(np.array(oh))
oh.columns= oh_encode_df.columns

time_end = time.time() 
timespend = (time_end- time_start)/60
print("時間: ", round(timespend, 2), " 分鐘")

時間:  6.84  分鐘


# Training & Testing

In [78]:
# Import Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# Accuracy
from sklearn import metrics
# 選出前三高的機率
import heapq
# Import train_test_split function
from sklearn.model_selection import train_test_split

In [79]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(oh, 
  data["Group"], test_size= 0.005, random_state= 109) 

# Modeling

In [80]:
# Create a Gaussian Classifier
mnb = MultinomialNB()
# Train the model using the training sets
mnb.fit(X_train, y_train)
# Predict the response for test dataset
y_pred = mnb.predict(X_test)

time_end = time.time() 
timespend = (time_end- time_start)/60
print("時間: ", round(timespend, 2), " 分鐘")

時間:  14.25  分鐘


# 前三高

In [81]:
all_prob= mnb.predict_proba(X_test)
all_prob= pd.DataFrame(all_prob)
all_prob.columns= range(1, 36)

prob3 = pd.DataFrame(columns= ["Prob1", "Prob2", "Prob3"], index= all_prob.index)

for i in all_prob.index :
    test = list(all_prob.loc[i])
    
    big3 = heapq.nlargest(3, test) # 前三高的數值
    big3_group = list(map(test.index, big3)) # 前三高的位置
    prob3.loc[i]= [i+1 for i in big3_group]

prob3['Correct'] = np.nan

for i in range(all_prob.shape[0]) :
    if list(y_test)[i] in list(prob3.loc[i]) :
        prob3['Correct'][i] = True
    else:
        prob3['Correct'][i] = False
        
time_end = time.time() 
timespend = (time_end- time_start)/60
print("時間: ", round(timespend, 2), " 分鐘")

時間:  14.26  分鐘


# Accuracy

In [82]:
Acc = metrics.accuracy_score(y_test, y_pred)
Acc3 = np.sum(prob3['Correct'] != 0) / all_prob.shape[0]

print("Acc1: ", Acc)
print("Acc3: ", Acc3)

Acc1:  0.23170731707317074
Acc3:  0.3878048780487805


# 選詞

In [83]:
groups = y_train.value_counts()
groups = pd.DataFrame(groups)
groups.columns = ["count"]
groups["prop"]= groups["count"] / groups["count"].sum(axis= 0)

In [84]:
all_prob= mnb.predict_proba(oh_encode_df)
all_prob= pd.DataFrame(all_prob)
all_prob.columns= range(1, 36)
sd= []

for i in all_prob.index :
    for j in range(1, 36) :
            all_prob[j][i]= all_prob[j][i]/ groups["prop"][j]

    max_val= max(all_prob.loc[i])
    all_prob.loc[i]= [k/max_val for k in all_prob.loc[i]]
    
    sd.append(np.std(all_prob.loc[i], ddof= 0))

all_prob.index= oh_encode_df.columns
all_prob.to_excel(PATH+ filename+ "all_prob.xlsx")
sd= pd.DataFrame(sd)
sd.index= oh_encode_df.columns
sd.columns= [filename]
sd= sd.sort_values(by= [filename])
sd.insert(0, 'Word', sd.index)
sd.to_excel(PATH+ filename+ "sd.xlsx", index= False)

time_end = time.time() 
timespend = (time_end- time_start)/60
print("時間: ", round(timespend, 2), " 分鐘")

時間:  18.81  分鐘
